In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
from pathlib import Path

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [2]:
# Load the data
file_path = "https://raw.githubusercontent.com/kwinterling/FinalProject/main/smoker_data/Resources/train_dataset.csv"
df = pd.read_csv(file_path)
#df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()


df.head()

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,35,170,85,97.0,0.9,0.9,1,1,118,78,...,70,142,19.8,1,1.0,61,115,125,1,1
1,20,175,110,110.0,0.7,0.9,1,1,119,79,...,71,114,15.9,1,1.1,19,25,30,1,0
2,45,155,65,86.0,0.9,0.9,1,1,110,80,...,57,112,13.7,3,0.6,1090,1400,276,0,0
3,45,165,80,94.0,0.8,0.7,1,1,158,88,...,46,91,16.9,1,0.9,32,36,36,0,0
4,20,165,60,81.0,1.5,0.1,1,1,109,64,...,47,92,14.9,1,1.2,26,28,15,0,0


In [3]:
df.dtypes

age                      int64
height(cm)               int64
weight(kg)               int64
waist(cm)              float64
eyesight(left)         float64
eyesight(right)        float64
hearing(left)            int64
hearing(right)           int64
systolic                 int64
relaxation               int64
fasting blood sugar      int64
Cholesterol              int64
triglyceride             int64
HDL                      int64
LDL                      int64
hemoglobin             float64
Urine protein            int64
serum creatinine       float64
AST                      int64
ALT                      int64
Gtp                      int64
dental caries            int64
smoking                  int64
dtype: object

In [4]:
# Drop the non-beneficial ID columns.
# new_df = df.drop(['waist(cm)', 'height(cm)', 'weight(kg)', 'eyesight(left)', 'eyesight(right)', 'hearing(left)', 'hearing(right)', 'Urine protein', 'dental caries', 'relaxation', 'AST', 'age', 'systolic'], axis=1)
new_df = df.drop(['eyesight(left)', 'height(cm)', 'eyesight(right)', 'hearing(left)', 'hearing(right)', 'Urine protein', 'dental caries', 'relaxation', 'AST', 'age', 'systolic'], axis=1)
# new_df = df
new_df.head()

,weight(kg),waist(cm),fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,serum creatinine,ALT,Gtp,smoking
0,85,97.0,97,239,153,70,142,19.8,1.0,115,125,1
1,110,110.0,88,211,128,71,114,15.9,1.1,25,30,0
2,65,86.0,80,193,120,57,112,13.7,0.6,1400,276,0
3,80,94.0,249,210,366,46,91,16.9,0.9,36,36,0
4,60,81.0,100,179,200,47,92,14.9,1.2,28,15,0


In [5]:
X = new_df.drop(columns="smoking")
y = pd.DataFrame(new_df["smoking"])

In [6]:
X.describe()

,weight(kg),waist(cm),fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,serum creatinine,ALT,Gtp
count,38984.000000,38984.000000,38984.000000,38984.000000,38984.000000,38984.000000,38984.000000,38984.000000,38984.000000,38984.000000,38984.000000
mean,65.938718,82.062115,99.342269,196.883491,126.749461,57.293146,115.081495,14.624264,0.886030,27.145188,39.905038
std,12.896581,9.326798,20.642741,36.353945,71.803143,14.617822,42.883163,1.566528,0.220621,31.309945,49.693843
min,30.000000,51.000000,46.000000,55.000000,8.000000,4.000000,1.000000,4.900000,0.100000,1.000000,2.000000
25%,55.000000,76.000000,89.000000,172.000000,74.000000,47.000000,91.000000,13.600000,0.800000,15.000000,17.000000
50%,65.000000,82.000000,96.000000,195.000000,108.000000,55.000000,113.000000,14.800000,0.900000,21.000000,26.000000
75%,75.000000,88.000000,104.000000,219.000000,160.000000,66.000000,136.000000,15.800000,1.000000,31.000000,44.000000
max,135.000000,129.000000,423.000000,445.000000,999.000000,359.000000,1860.000000,21.100000,11.600000,2914.000000,999.000000


In [7]:
y['smoking'].value_counts()

0    24666
1    14318
Name: smoking, dtype: int64

In [8]:
from collections import Counter
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train.shape

(29238, 11)

In [9]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(29238, 11)
(9746, 11)
(29238, 1)
(9746, 1)


In [10]:
Counter(y_train)

Counter({'smoking': 1})

In [11]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.datasets import make_classification

In [12]:
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=200, max_depth=32, random_state=1)
brf

BalancedRandomForestClassifier(max_depth=32, n_estimators=200, random_state=1)

In [13]:
brf.fit(X_train_scaled, y_train)
y_pred = brf.predict(X_test_scaled)

<ipython-input-13-aff86d4fa7d9>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  brf.fit(X_train_scaled, y_train)


In [14]:
rf = BalancedRandomForestClassifier()
parameters = {
    "n_estimators": [5, 20, 40, 80, 100, 200],
    "max_depth": [2, 4, 8, 16, 32, None]
}

cv = GridSearchCV(rf, parameters, cv=5)
cv.fit(X_train.values, y_train.values.ravel())

GridSearchCV(cv=5, estimator=BalancedRandomForestClassifier(),
             param_grid={'max_depth': [2, 4, 8, 16, 32, None],
                         'n_estimators': [5, 20, 40, 80, 100, 200]})

In [15]:
print(cv.best_params_)

{'max_depth': 32, 'n_estimators': 80}


In [16]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = brf.predict(X_test_scaled)
y_pred

array([1, 1, 1, ..., 0, 1, 0])

In [17]:
balanced_accuracy_score(y_test, y_pred)

0.7873855422212335

In [18]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = brf.predict(X_test_scaled)
confusion_matrix(y_test, y_pred)

array([[4512, 1680],
       [ 547, 3007]])

In [19]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.89      0.73      0.85      0.80      0.79      0.61      6192
          1       0.64      0.85      0.73      0.73      0.79      0.62      3554

avg / total       0.80      0.77      0.80      0.78      0.79      0.61      9746



In [20]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_
importances
sorted(zip(brf.feature_importances_, X.columns), reverse=True)

[(0.18409661880553777, 'hemoglobin'),
 (0.1293358823569794, 'Gtp'),
 (0.09509001058359914, 'triglyceride'),
 (0.07912218995841395, 'waist(cm)'),
 (0.07814212176591824, 'LDL'),
 (0.07722994814821531, 'Cholesterol'),
 (0.07428405358028004, 'ALT'),
 (0.07406437641358742, 'HDL'),
 (0.07269753994172266, 'fasting blood sugar'),
 (0.07157344989488999, 'weight(kg)'),
 (0.06436380855085601, 'serum creatinine')]

In [21]:
from sklearn.ensemble import GradientBoostingClassifier

xgb = GradientBoostingClassifier()

xgb.fit(X_train_scaled, y_train.values.ravel())
y_pd = xgb.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pd)


0.7132804922647844